# Import Libraries

In [265]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz

print('Pandas version', pd.__version__)
print('Numpy version', np.__version__)

Pandas version 0.20.3
Numpy version 1.14.0


# Read in data

## Read in school data

In [313]:
schools = pd.read_pickle('../data/schools_crimes.pkl')

In [314]:
budget = pd.read_pickle('../data/budget_raw.pkl')

In [315]:
budget.head()

,School,2016-17,2015-16,Pct. Change,New Budget,Budget Change
0,Chicago Vocational High School,922,978,-5.73%,"$8,316,708","-$235,904"
1,Dunbar High School,563,761,-26.02%,"$6,216,418","-$88,989"
2,Jones High School,1820,1670,8.98%,"$11,061,717","-$231,475"
3,Prosser High School,1381,1455,-5.09%,"$10,986,115","-$156,831"
4,Payton High School,981,887,10.60%,"$6,720,149","-$109,332"


In [316]:
schools.head()

,index,School_ID,Short_Name,Long_Name,School_Type,Zip,Facebook,Twitter,Student_Count_Total,Student_Count_Low_Income,...,School_Longitude,Average_ACT_School,Graduation_Rate_School,fb_likes,tw_followers,tw_tweets,crime_counts,crime_outlier,crimes_per_student,crimes_per_1000_students
0,100,400098,NOBLE - MUCHIN HS,Noble - Muchin College Prep,Charter,60602,http://www.facebook.com/BeNobleMuchin,https://twitter.com/BeNobleMuchin,964,804,...,-87.626338,21.8,86.4,600.0,287.0,421.0,78,True,0.080913,80.912863
1,91,609678,JONES HS,William Jones College Preparatory High School,Selective enrollment,60605,https://www.facebook.com/JonesCollegePrepHS/,https://twitter.com/jonescollegehs,1866,732,...,-87.627675,26.2,91.8,NaN,455.0,549.0,78,True,0.041801,41.800643
2,8,400056,NOBLE - ROWE CLARK HS,Noble - Rowe-Clark Math and Science Academy,Charter,60651,https://www.facebook.com/BeNobleRCMSA/,https://twitter.com/BeNobleRCMSA,474,441,...,-87.718047,18.5,82.4,427.0,187.0,205.0,333,True,0.702532,702.531646
3,10,400058,NORTH LAWNDALE - CHRISTIANA HS,North Lawndale College Prep - Christiana,Charter,60623,https://www.facebook.com/nlcphs,NaN,357,349,...,-87.708987,15.3,67.1,1419.0,NaN,NaN,229,True,0.641457,641.456583
4,66,610389,ORR HS,Orr Academy High School,Neighborhood,60624,https://www.facebook.com/orracademyhighschool,https://twitter.com/orracademy,276,263,...,-87.726166,14.2,39.0,424.0,213.0,70.0,199,True,0.721014,721.014493


## Strip 'High School' from the names of schools in each dataset

In [317]:
def clean_school_name(df, name_col):
    """
    Takes in a dataframe and column name to be cleaned and returns a standard format to be used in name matching
    """
    
    clean_names = []
    
    for name in df[name_col]:
        
        rm_hs = name.replace('High School', '').replace('HS', '').strip()
        cleaned = rm_hs.translate(str.maketrans("", "", ",.-'\"():;+/?$°@"))
        clean_names.append(cleaned.lower())
        
    return clean_names

In [318]:
schools['cleaned_name'] = clean_school_name(schools, 'Long_Name')
budget['cleaned_name'] = clean_school_name(budget, 'School')

## Merge schools and budgets on fuzzy string match

In [319]:
def find_best_matches(df_1, df_2, match_col, cutoff=80, scorer=fuzz.token_set_ratio):
    
    best_matches = []
    
    for index, row in df_1.iterrows():
    
        match = process.extractOne(
                        df_1.loc[index, match_col], 
                        choices=df_2.loc[:, match_col], 
                        scorer=scorer,
                        score_cutoff=cutoff)
        
        best_matches.append(match)

    return best_matches

I'm using the fuzzy wuzzy package's token_set_ratio method because it will subset each string into it's component parts, find the components that are contained in the set of target components and then score the remainder (order does not matter)

Example:
```python
>>> fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")
    84
>>> fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")
    100
```
https://github.com/seatgeek/fuzzywuzzy

In [320]:
schools['matches'] = find_best_matches(schools, budget, 'cleaned_name', cutoff=90)
schools['match_id'] = [int(x[2]) if x is not None else None for x in schools['matches']]

In [321]:
schools.head()

,index,School_ID,Short_Name,Long_Name,School_Type,Zip,Facebook,Twitter,Student_Count_Total,Student_Count_Low_Income,...,fb_likes,tw_followers,tw_tweets,crime_counts,crime_outlier,crimes_per_student,crimes_per_1000_students,cleaned_name,matches,match_id
0,100,400098,NOBLE - MUCHIN HS,Noble - Muchin College Prep,Charter,60602,http://www.facebook.com/BeNobleMuchin,https://twitter.com/BeNobleMuchin,964,804,...,600.0,287.0,421.0,78,True,0.080913,80.912863,noble muchin college prep,None,NaN
1,91,609678,JONES HS,William Jones College Preparatory High School,Selective enrollment,60605,https://www.facebook.com/JonesCollegePrepHS/,https://twitter.com/jonescollegehs,1866,732,...,NaN,455.0,549.0,78,True,0.041801,41.800643,william jones college preparatory,"(jones, 100, 2)",2.0
2,8,400056,NOBLE - ROWE CLARK HS,Noble - Rowe-Clark Math and Science Academy,Charter,60651,https://www.facebook.com/BeNobleRCMSA/,https://twitter.com/BeNobleRCMSA,474,441,...,427.0,187.0,205.0,333,True,0.702532,702.531646,noble roweclark math and science academy,None,NaN
3,10,400058,NORTH LAWNDALE - CHRISTIANA HS,North Lawndale College Prep - Christiana,Charter,60623,https://www.facebook.com/nlcphs,NaN,357,349,...,1419.0,NaN,NaN,229,True,0.641457,641.456583,north lawndale college prep christiana,"(lawndale, 100, 246)",246.0
4,66,610389,ORR HS,Orr Academy High School,Neighborhood,60624,https://www.facebook.com/orracademyhighschool,https://twitter.com/orracademy,276,263,...,424.0,213.0,70.0,199,True,0.721014,721.014493,orr academy,"(orr, 100, 467)",467.0


### Checking if budget rows have been duplicated
Let's make sure there aren't multiple schools matching to a single budget entry

In [322]:
distinct_matches = schools.groupby('match_id')['School_ID'].nunique()
distinct_matches.sort_values(ascending=False).head()

match_id
49.0     6
246.0    3
41.0     2
445.0    2
28.0     1
Name: School_ID, dtype: int64

Oofph, I need to investigate the first 4 matches there... let's add a column with these match counts for every match_id so I can look at them in more detail

In [323]:
schools['school_match_count'] = [distinct_matches[x] if not np.isnan(x) else None for x in schools['match_id']]

### Checking schools with no budget rows
Which schools were not matched with a budget row

In [324]:
unmatched_schools = schools.loc[schools['match_id'].isnull()]

In [325]:
len(unmatched_schools)

32

In [326]:
unmatched_schools['cleaned_name']

0                           noble  muchin college prep
2            noble  roweclark math and science academy
5                    austin college and career academy
12                   noble  chicago bulls college prep
17                          noble  golder college prep
21                        consuella b york alternative
22                                 cics  ralph ellison
29                                epic academy charter
31                    perspectives  leadership academy
32                       chicago  for the arts chiarts
35                aspira charter school  early college
41                       perspectives  rodney d joslin
43                                 ogden international
46              perspectives  math and science academy
50                           noble  noble college prep
51                                      cics  longwood
54                      chicago virtual charter school
56                          noble  rauner college prep
57        

Let's merge what we can and then export the file for manual investigation (to get to MVP)

In [328]:
matches = pd.merge(schools, budget, how='outer', 
                  left_on='match_id', right_index=True)

In [308]:
print(len(matches))
matches.columns

542


Index(['index', 'School_ID', 'Short_Name', 'Long_Name', 'School_Type', 'Zip',
       'Facebook', 'Twitter', 'Student_Count_Total',
       'Student_Count_Low_Income', 'School_Hours', 'Transportation_El',
       'School_Latitude', 'School_Longitude', 'Average_ACT_School',
       'Graduation_Rate_School', 'fb_likes', 'tw_followers', 'tw_tweets',
       'crime_counts', 'crime_outlier', 'crimes_per_student',
       'crimes_per_1000_students', 'cleaned_name_x', 'matches', 'match_id',
       'school_match_count', 'School', '2016-17', '2015-16', 'Pct. Change',
       'New Budget', 'Budget Change', 'cleaned_name_y'],
      dtype='object')

## Manually resolve as best as possible

In [330]:
matches_key = matches[['School_ID', 'Short_Name', 'Long_Name', 'Zip', 'Student_Count_Total',
                      'cleaned_name_x', 'match_id', 'School', 'cleaned_name_y', '2016-17', 'school_match_count']]

In [331]:
matches_key.to_csv('../data/school_budget_manual.csv', index=False)

## Merge Together

Here's where I go manually try to resolve the ~30 unmatched schools against the budget records... 

In [332]:
# Read in new match key
match_key = pd.read_csv('../data/school_budget_manual_key.csv', encoding = "ISO-8859-1")

In [333]:
count_resolved = len(match_key[(~match_key['match_id'].isnull()) & (~match_key['School_ID'].isnull())])
print('I was able to resolve {} schools to budget records'.format(count_resolved))

I was able to resolve 90 schools to budget records


Let's try running the model on these observations to see if there is any correlation - then we can go back and add in more observations or resolve the missing data somehow

In [336]:
match_key.head()

,School_ID,Short_Name,Long_Name,Zip,Student_Count_Total,cleaned_name_x,match_id,School,cleaned_name_y,2016-17,school_match_count
0,609764.0,JUAREZ HS,Benito Juarez Community Academy High School,60608.0,1739.0,benito juarez community academy,55.0,Juarez High School,juarez,1610,1.0
1,400054.0,NOBLE - PRITZKER HS,Noble - Pritzker College Prep,60639.0,992.0,noble pritzker college prep,393.0,Pritzker,pritzker,707,1.0
2,609726.0,BROOKS HS,Gwendolyn Brooks College Preparatory Academy HS,60628.0,959.0,gwendolyn brooks college preparatory academy,30.0,Brooks High School,brooks,916,1.0
3,400094.0,EPIC HS,EPIC Academy Charter High School,60617.0,523.0,epic academy charter,NaN,NaN,NaN,NaN,NaN
4,610518.0,AUSTIN CCA HS,Austin College and Career Academy High School,60644.0,223.0,austin college and career academy,NaN,NaN,NaN,NaN,NaN


In [339]:
school_matches = pd.merge(schools, match_key.loc[:,['School_ID', 'match_id']], how='inner', on='School_ID')
all_matches = pd.merge(school_matches, budget, how='inner', left_on='match_id_y', right_index=True)

In [340]:
all_matches.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90 entries, 1 to 120
Data columns (total 35 columns):
index                       90 non-null int64
School_ID                   90 non-null object
Short_Name                  90 non-null object
Long_Name                   90 non-null object
School_Type                 90 non-null object
Zip                         90 non-null int64
Facebook                    41 non-null object
Twitter                     54 non-null object
Student_Count_Total         90 non-null int64
Student_Count_Low_Income    90 non-null int64
School_Hours                88 non-null object
Transportation_El           68 non-null object
School_Latitude             90 non-null float64
School_Longitude            90 non-null float64
Average_ACT_School          90 non-null float64
Graduation_Rate_School      90 non-null float64
fb_likes                    35 non-null float64
tw_followers                54 non-null float64
tw_tweets                   54 non-null float64


# Creating budget and demographic dimensions

## Budget Dimensions

Before we can identify a good budget feature like 'dollars per student' we need to convert the budget object to integers

In [357]:
all_matches.reset_index(drop=True, inplace=True)
all_matches.head()

,index,School_ID,Short_Name,Long_Name,School_Type,Zip,Facebook,Twitter,Student_Count_Total,Student_Count_Low_Income,...,match_id_x,school_match_count,match_id_y,School,2016-17,2015-16,Pct. Change,New Budget,Budget Change,cleaned_name_y
0,91,609678,JONES HS,William Jones College Preparatory High School,Selective enrollment,60605,https://www.facebook.com/JonesCollegePrepHS/,https://twitter.com/jonescollegehs,1866,732,...,2.0,1.0,2,Jones High School,1820,1670,8.98%,"$11,061,717","-$231,475",jones
1,10,400058,NORTH LAWNDALE - CHRISTIANA HS,North Lawndale College Prep - Christiana,Charter,60623,https://www.facebook.com/nlcphs,NaN,357,349,...,246.0,3.0,246,Lawndale,243,329,-26.14%,"$1,850,711","-$411,432",lawndale
2,115,400059,NORTH LAWNDALE - COLLINS HS,North Lawndale College Prep - Collins,Charter,60623,https://www.facebook.com/nlcphs,NaN,351,340,...,246.0,3.0,246,Lawndale,243,329,-26.14%,"$1,850,711","-$411,432",lawndale
3,66,610389,ORR HS,Orr Academy High School,Neighborhood,60624,https://www.facebook.com/orracademyhighschool,https://twitter.com/orracademy,276,263,...,467.0,1.0,467,Orr High School,396,405,-2.22%,"$3,702,822","$305,624",orr
4,9,610394,UPLIFT HS,Uplift Community High School,Small,60640,NaN,https://twitter.com/UpliftTitans,245,229,...,471.0,1.0,471,Uplift High School,301,311,-3.22%,"$2,633,463","$91,618",uplift


In [364]:
def make_int(series):
    
    stripped_str = [int(x.replace('$', '').replace(',','')) for x in series]
    
    return stripped_str
    

In [371]:
def dollars_per_student(budget_col, student_col):
    
    dollars_int = make_int(series=budget_col)
    
    return [ dollars/int(students) for (dollars, students) in zip(dollars_int, student_col)]

In [372]:
all_matches['dollars_per_student'] = dollars_per_student(budget_col=all_matches['New Budget'],
                                                        student_col=all_matches['Student_Count_Total'])

Now to calculate a percent diff in the budget from the previous year (we are given the change amount)

In [381]:
def budget_percent_diff(current_budget, budget_diff):
    
    budget_int = make_int(current_budget)
    diff_int = make_int(budget_diff)
    
    last_year_budget = [current - change for (current, change) in zip(budget_int, diff_int)]
    
    percent_diff = [diff/last_year for (last_year, diff) in zip(last_year_budget, diff_int)]
    
    return percent_diff
    

In [376]:
all_matches['last_year'] = budget_percent_diff(current_budget=all_matches['New Budget'],
                                              budget_diff=all_matches['Budget Change'])

In [382]:
all_matches['percent_diff_budget'] = budget_percent_diff(current_budget=all_matches['New Budget'],
                                                          budget_diff=all_matches['Budget Change'])

## Student Dimensions

I need to find the percentage of students who are low income and the average ACT score

In [384]:
all_matches.head()

,index,School_ID,Short_Name,Long_Name,School_Type,Zip,Facebook,Twitter,Student_Count_Total,Student_Count_Low_Income,...,School,2016-17,2015-16,Pct. Change,New Budget,Budget Change,cleaned_name_y,dollars_per_student,last_year,percent_diff_budget
0,91,609678,JONES HS,William Jones College Preparatory High School,Selective enrollment,60605,https://www.facebook.com/JonesCollegePrepHS/,https://twitter.com/jonescollegehs,1866,732,...,Jones High School,1820,1670,8.98%,"$11,061,717","-$231,475",jones,5928.036977,11293192.0,-0.020497
1,10,400058,NORTH LAWNDALE - CHRISTIANA HS,North Lawndale College Prep - Christiana,Charter,60623,https://www.facebook.com/nlcphs,NaN,357,349,...,Lawndale,243,329,-26.14%,"$1,850,711","-$411,432",lawndale,5184.064426,2262143.0,-0.181877
2,115,400059,NORTH LAWNDALE - COLLINS HS,North Lawndale College Prep - Collins,Charter,60623,https://www.facebook.com/nlcphs,NaN,351,340,...,Lawndale,243,329,-26.14%,"$1,850,711","-$411,432",lawndale,5272.680912,2262143.0,-0.181877
3,66,610389,ORR HS,Orr Academy High School,Neighborhood,60624,https://www.facebook.com/orracademyhighschool,https://twitter.com/orracademy,276,263,...,Orr High School,396,405,-2.22%,"$3,702,822","$305,624",orr,13416.021739,3397198.0,0.089964
4,9,610394,UPLIFT HS,Uplift Community High School,Small,60640,NaN,https://twitter.com/UpliftTitans,245,229,...,Uplift High School,301,311,-3.22%,"$2,633,463","$91,618",uplift,10748.828571,2541845.0,0.036044


In [385]:
all_matches['percent_low_income'] = [low_income/all_students for (all_students, low_income) 
                                     in zip(all_matches['Student_Count_Total'], all_matches['Student_Count_Low_Income'])]

In [388]:
all_matches.loc[:10, ['Student_Count_Total', 'Student_Count_Low_Income', 'percent_low_income']]

,Student_Count_Total,Student_Count_Low_Income,percent_low_income
0,1866,732,0.392283
1,357,349,0.977591
2,351,340,0.968661
3,276,263,0.952899
4,245,229,0.934694
5,313,295,0.942492
6,113,105,0.929204
7,598,492,0.822742
8,731,703,0.961696
9,218,196,0.899083


In [389]:
list(all_matches)

['index',
 'School_ID',
 'Short_Name',
 'Long_Name',
 'School_Type',
 'Zip',
 'Facebook',
 'Twitter',
 'Student_Count_Total',
 'Student_Count_Low_Income',
 'School_Hours',
 'Transportation_El',
 'School_Latitude',
 'School_Longitude',
 'Average_ACT_School',
 'Graduation_Rate_School',
 'fb_likes',
 'tw_followers',
 'tw_tweets',
 'crime_counts',
 'crime_outlier',
 'crimes_per_student',
 'crimes_per_1000_students',
 'cleaned_name_x',
 'matches',
 'match_id_x',
 'school_match_count',
 'match_id_y',
 'School',
 '2016-17',
 '2015-16',
 'Pct. Change',
 'New Budget',
 'Budget Change',
 'cleaned_name_y',
 'dollars_per_student',
 'last_year',
 'percent_diff_budget',
 'percent_low_income']

In [391]:
cleaned_data = all_matches.loc[:,['School_ID','Graduation_Rate_School', 'Short_Name', 'School_Type','Transportation_El',
                                      'School_Latitude','School_Longitude','Average_ACT_School', 'crimes_per_student',
                                      'dollars_per_student','percent_diff_budget', 'percent_low_income']]

In [392]:
cleaned_data.head()

,School_ID,Graduation_Rate_School,Short_Name,School_Type,Transportation_El,School_Latitude,School_Longitude,Average_ACT_School,crimes_per_student,dollars_per_student,percent_diff_budget,percent_low_income
0,609678,91.8,JONES HS,Selective enrollment,"Blue, Brown, Pink, Red",41.873066,-87.627675,26.2,0.041801,5928.036977,-0.020497,0.392283
1,400058,67.1,NORTH LAWNDALE - CHRISTIANA HS,Charter,Green,41.858564,-87.708987,15.3,0.641457,5184.064426,-0.181877,0.977591
2,400059,69.7,NORTH LAWNDALE - COLLINS HS,Charter,Red,41.864146,-87.700681,14.5,0.150997,5272.680912,-0.181877,0.968661
3,610389,39.0,ORR HS,Neighborhood,NaN,41.894449,-87.726166,14.2,0.721014,13416.021739,0.089964,0.952899
4,610394,74.3,UPLIFT HS,Small,Red,41.965547,-87.652473,15.6,0.714286,10748.828571,0.036044,0.934694


# Pickle Data

In [393]:
cleaned_data.to_pickle('../data/cleaned_data.pkl')